In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [17]:
MODELPATH = './models/model.h5'

In [2]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

In [4]:
data = pd.read_csv('fer2013.csv')

In [7]:
pixels = data['pixels'].tolist() 
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] 
    face = np.asarray(face).reshape(width, height)
    
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion']).values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [10]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 128)      

In [12]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [23]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          )

Train on 29068 samples, validate on 3589 samples
Epoch 1/100
29068/29068 [==============================] - 69s 2ms/step - loss: 2.0309 - accuracy: 0.2121 - val_loss: 1.8110 - val_accuracy: 0.2407
Epoch 2/100
29068/29068 [==============================] - 60s 2ms/step - loss: 1.8222 - accuracy: 0.2530 - val_loss: 1.7427 - val_accuracy: 0.2884
Epoch 3/100
29068/29068 [==============================] - 60s 2ms/step - loss: 1.7535 - accuracy: 0.2894 - val_loss: 1.8621 - val_accuracy: 0.2329
Epoch 4/100
29068/29068 [==============================] - 60s 2ms/step - loss: 1.6669 - accuracy: 0.3366 - val_loss: 1.6534 - val_accuracy: 0.3580
Epoch 5/100
29068/29068 [==============================] - 60s 2ms/step - loss: 1.5723 - accuracy: 0.3832 - val_loss: 1.5619 - val_accuracy: 0.3675
Epoch 6/100
29068/29068 [==============================] - 60s 2ms/step - loss: 1.4921 - accuracy: 0.4142 - val_loss: 1.4895 - val_accuracy: 0.3987
Epoch 7/100
29068/29068 [==============================] - 60s 

Epoch 56/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5803 - accuracy: 0.8017 - val_loss: 1.1939 - val_accuracy: 0.6408
Epoch 57/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5838 - accuracy: 0.8019 - val_loss: 1.1443 - val_accuracy: 0.6484
Epoch 58/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5626 - accuracy: 0.8095 - val_loss: 1.1496 - val_accuracy: 0.6400
Epoch 59/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5518 - accuracy: 0.8104 - val_loss: 1.1495 - val_accuracy: 0.6473
Epoch 60/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5410 - accuracy: 0.8162 - val_loss: 1.1601 - val_accuracy: 0.6453
Epoch 61/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5431 - accuracy: 0.8139 - val_loss: 1.2098 - val_accuracy: 0.6230
Epoch 62/100
29068/29068 [==============================] - 60s 2ms/step - loss: 0.5343 - accuracy: 0.8214

In [24]:
model.save('model.h5')